In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [ ]:
data = pd.read_excel('dataset.xlsx')
data.head()

,Unnamed: 0,id,name,artist,disc,duration,episode,explicit,popularity,danceability,...,energy,instrumentalness,liveness,loudness,mode,speechiness,tempo,time_signature,valence,lyrics
0,0,1hA4856JVAa0qqgKg6olJf,"The Messenger - Live from Las Vegas, 2011",Linkin Park,1,233280,False,False,37,0.325,...,0.409,0.000013,0.1880,-5.711,1,0.0295,109.001,4,0.187,6 ContributorsThe Messenger (Live from Las Veg...
1,1,0GgN4MhR5GKn5IcKN0e0rG,Cancer,My Chemical Romance,1,142973,False,False,72,0.457,...,0.515,0.000000,0.2780,-4.310,1,0.0261,74.984,4,0.222,93 ContributorsCancer Lyrics\nTurn away\nIf yo...
2,2,10nyNJ6zNy2YVYLrcwLccB,No Surprises,Radiohead,1,229120,False,False,83,0.255,...,0.393,0.003610,0.1130,-10.654,1,0.0278,76.426,4,0.118,129 ContributorsTranslationsРусскийDeutschEspa...
3,3,7lRlq939cDG4SzWOF4VAnd,I'm Not Okay (I Promise),My Chemical Romance,1,186480,False,True,78,0.210,...,0.940,0.000000,0.2690,-3.427,1,0.1230,179.722,4,0.255,109 ContributorsTranslationsEspañolI’m Not Oka...
4,4,5u2FOoFhp495GIj5BJC77J,Hold On Till May,Pierce The Veil,1,278586,False,False,66,0.460,...,0.910,0.001550,0.0735,-3.710,1,0.0525,73.019,4,0.327,58 ContributorsHold on Till May Lyrics\nShe si...


In [ ]:
clean_lyrics = lambda x: x[x.find('Lyrics')+len('Lyrics'):] if x.find('Lyrics') != -1 else x
data['lyrics_limpio'] = data['lyrics'].apply(clean_lyrics)


0       When you feel you're alone, cut off from this ...
1       \nTurn away\nIf you could, get me a drink\nOf ...
2       \nA heart that's full up like a landfill\nA jo...
3       \n\nWell, if you wanted honesty\nThat's all yo...
4       \nShe sits up high, surrounded by the sun\nOne...
                              ...                        
1029    \n\nMe trae enamorado sin saber su nombre (Yea...
1030    \n\nQuién crees que eres usted maltratándome a...
1031    \n\nUgly Duck, El Jordan veintitr-, woh\n(Nes ...
1032    \n\nDe Psycho Bunny o de Burberry, fumando pur...
1033    \n\nDesde que tú te fuiste puse el modo puto e...
Name: lyrics_limpio, Length: 1034, dtype: object

In [ ]:
### Encontrar el lenguaje

#!pip install langdetect
from langdetect import detect

In [ ]:
texto = '6 ContributorsThe Messenger (Live fromLyricsA partir de acá empieza la letra'

texto[texto.find('Lyrics') + len('Lyrics'):]

'A partir de acá empieza la letra'

In [ ]:
data['lang'] = data['lyrics_limpio'].apply(detect)

In [ ]:
#!pip install unidecode
from unidecode import unidecode

data['lyrics_decode'] = data['lyrics_limpio'].apply(unidecode)

In [ ]:
data['lyrics_decode'] = data['lyrics_decode'].str.lower()

In [ ]:
### Solamente inglés, francés y español
data = data[data['lang'].isin(['en','fr','es'])]



In [ ]:
#!pip install nltk
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer